In [21]:
from ortools.linear_solver import pywraplp
from weasyprint import HTML, CSS
import tempfile
import os

In [1]:
data = {
    "instructors": [
        {"name": "AEL", "lessons": ["Math101"], "preferred_days": ["Tuesday", "Wednesday", "Thursday", "Friday"]},
        {
            "name": "BANUDIRI",
            "lessons": ["Math101", "Chemistry301"],
            "preferred_days": ["Tuesday", "Wednesday", "Friday"],
        },
        {
            "name": "JaneSmith",
            "lessons": ["Biology101", "Philosophy101"],
            "preferred_days": ["Tuesday", "Thursday", "Friday"],
        },
        {
            "name": "JohnDoe",
            "lessons": ["Physics201", "Philosophy101"],
            "preferred_days": ["Tuesday", "Thursday", "Friday"],
        },
        {
            "name": "AliceJohnson",
            "lessons": ["English101", "History201"],
            "preferred_days": ["Monday", "Tuesday", "Thursday"],
        },
        {
            "name": "RobertBrown",
            "lessons": ["Philosophy101", "Math102"],
            "preferred_days": ["Monday", "Wednesday", "Friday"],
        },
        {
            "name": "EmmaThomas",
            "lessons": ["Spanish101", "French101"],
            "preferred_days": ["Monday", "Wednesday", "Friday"],
        },
        {
            "name": "MichaelChen",
            "lessons": ["ComputerScience101", "Math103"],
            "preferred_days": ["Tuesday", "Thursday"],
        },
        {
            "name": "SarahPatel",
            "lessons": ["Art101", "History202"],
            "preferred_days": ["Monday", "Wednesday", "Friday"],
        },
        {
            "name": "DavidKim",
            "lessons": ["Music101", "Drama101"],
            "preferred_days": ["Tuesday", "Thursday", "Friday"],
        },
        {"name": "LauraWilson", "lessons": ["Chemistry301"], "preferred_days": ["Tuesday"]},
    ],
    "lessons": [
        {"name": "Math101", "instructors": ["AEL", "BANUDIRI"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "Biology101", "instructors": ["JaneSmith"], "grade": 2, "type": "Online", "duration": 3},
        {"name": "English101", "instructors": ["AliceJohnson"], "grade": 4, "type": "FaceToFace", "duration": 2},
        {"name": "Physics201", "instructors": ["JohnDoe"], "grade": 2, "type": "FaceToFace", "duration": 3},
        {
            "name": "Philosophy101",
            "instructors": ["RobertBrown", "JohnDoe", "JaneSmith"],
            "grade": 1,
            "type": "FaceToFace",
            "duration": 1,
        },
        {"name": "History201", "instructors": ["AliceJohnson"], "grade": 2, "type": "Hybrid", "duration": 2},
        {"name": "Math102", "instructors": ["RobertBrown"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {
            "name": "Chemistry301",
            "instructors": ["LauraWilson", "BANUDIRI"],
            "grade": 3,
            "type": "Online",
            "duration": 3,
        },
        {
            "name": "Spanish101",
            "instructors": ["EmmaThomas"],
            "grade": 2,
            "type": "FaceToFace",
            "duration": 2,
        },
        {
            "name": "French101",
            "instructors": ["EmmaThomas"],
            "grade": 3,
            "type": "Hybrid",
            "duration": 2,
        },
        {
            "name": "ComputerScience101",
            "instructors": ["MichaelChen"],
            "grade": 4,
            "type": "FaceToFace",
            "duration": 2,
        },
        {
            "name": "Math103",
            "instructors": ["MichaelChen"],
            "grade": 3,
            "type": "Online",
            "duration": 2,
        },
        {
            "name": "Art101",
            "instructors": ["SarahPatel"],
            "grade": 1,
            "type": "FaceToFace",
            "duration": 2,
        },
        {
            "name": "History202",
            "instructors": ["SarahPatel"],
            "grade": 4,
            "type": "Hybrid",
            "duration": 2,
        },
        {
            "name": "Music101",
            "instructors": ["DavidKim"],
            "grade": 1,
            "type": "FaceToFace",
            "duration": 1,
        },
        {
            "name": "Drama101",
            "instructors": ["DavidKim"],
            "grade": 2,
            "type": "FaceToFace",
            "duration": 2,
        },
    ],
    "classrooms": [
        "MusicRoom301",
        "ArtStudio302",
        "LanguageLab303",
        "ComputerLab304",
        "Room101",
        "Lab202",
        "Room103",
        "Lab204",
        "Room205",
        "Lab206",
        "Room207",
        "Lab208",
    ],
}



In [2]:
def get_instructor_by_name(instructors, name):
    for instructor in instructors:
        if instructor["name"] == name:
            return instructor
    return None

In [3]:
def solve_schedule(data):
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return

    # Input data
    instructors = data["instructors"]
    lessons = data["lessons"]
    classrooms = data["classrooms"]

    time_slots = list(range(8, 17))  # 8:00 AM to 4:00 PM
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]

    # Decision variables - only for start times
    x = {}
    for lesson in lessons:
        for instructor in lesson["instructors"]:
            instructor_days = get_instructor_by_name(instructors, instructor)["preferred_days"]
            for day in instructor_days:
                # Only create variables for valid start times
                for time in range(8, 17 - lesson["duration"]):
                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            x[(lesson["name"], instructor, classroom, day, time)] = solver.BoolVar(
                                f'x_{lesson["name"]}_{instructor}_{classroom}_{day}_{time}'
                            )
                    else:  # Online classes don't need classrooms
                        x[(lesson["name"], instructor, None, day, time)] = solver.BoolVar(
                            f'x_{lesson["name"]}_{instructor}_None_{day}_{time}'
                        )

    # Constraint 1: Each lesson must be scheduled exactly once
    for lesson in lessons:
        constraint_expr = []
        for instructor in lesson["instructors"]:
            instructor_days = get_instructor_by_name(instructors, instructor)["preferred_days"]
            for day in instructor_days:
                for time in range(8, 17 - lesson["duration"]):
                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            if (lesson["name"], instructor, classroom, day, time) in x:
                                constraint_expr.append(x[(lesson["name"], instructor, classroom, day, time)])
                    else:
                        if (lesson["name"], instructor, None, day, time) in x:
                            constraint_expr.append(x[(lesson["name"], instructor, None, day, time)])
        solver.Add(sum(constraint_expr) == 1)

    # Constraint 2: No instructor can teach during overlapping time slots
    for instructor in instructors:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if instructor["name"] in lesson["instructors"]:
                        # Check all possible start times that would overlap with current time slot
                        for start_time in range(
                            max(8, time - lesson["duration"] + 1), min(time + 1, 17 - lesson["duration"])
                        ):
                            if lesson["type"] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (lesson["name"], instructor["name"], classroom, day, start_time) in x:
                                        constraint_expr.append(
                                            x[(lesson["name"], instructor["name"], classroom, day, start_time)]
                                        )
                            else:
                                if (lesson["name"], instructor["name"], None, day, start_time) in x:
                                    constraint_expr.append(
                                        x[(lesson["name"], instructor["name"], None, day, start_time)]
                                    )
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 3: No classroom can have overlapping classes
    for classroom in classrooms:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if lesson["type"] in ["FaceToFace", "Hybrid"]:
                        # Check all possible start times that would overlap with current time slot
                        for start_time in range(
                            max(8, time - lesson["duration"] + 1), min(time + 1, 17 - lesson["duration"])
                        ):
                            for instructor in lesson["instructors"]:
                                if (lesson["name"], instructor, classroom, day, start_time) in x:
                                    constraint_expr.append(x[(lesson["name"], instructor, classroom, day, start_time)])
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 4: No overlapping classes for the same grade level
    for grade in grades:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                grade_lessons = [lesson for lesson in lessons if lesson["grade"] == grade]
                for lesson in grade_lessons:
                    # Check all possible start times that would overlap with current time slot
                    for start_time in range(
                        max(8, time - lesson["duration"] + 1), min(time + 1, 17 - lesson["duration"])
                    ):
                        for instructor in lesson["instructors"]:
                            if lesson["type"] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (lesson["name"], instructor, classroom, day, start_time) in x:
                                        constraint_expr.append(
                                            x[(lesson["name"], instructor, classroom, day, start_time)]
                                        )
                            else:
                                if (lesson["name"], instructor, None, day, start_time) in x:
                                    constraint_expr.append(x[(lesson["name"], instructor, None, day, start_time)])
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Solve
    status = solver.Solve()

    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print("\nOptimal schedule found:\n")
        for lesson in lessons:
            for instructor in lesson["instructors"]:
                for day in days_of_week:
                    for time in range(8, 17 - lesson["duration"]):
                        if lesson["type"] in ["FaceToFace", "Hybrid"]:
                            for classroom in classrooms:
                                if (lesson["name"], instructor, classroom, day, time) in x:
                                    if x[(lesson["name"], instructor, classroom, day, time)].solution_value() == 1:
                                        print(
                                            f"Lesson: {lesson['name']} (Grade {lesson['grade']}) - Instructor: {instructor}"
                                        )
                                        print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                        print(f"Type: {lesson['type']}, Classroom: {classroom}\n")
                        else:
                            if (lesson["name"], instructor, None, day, time) in x:
                                if x[(lesson["name"], instructor, None, day, time)].solution_value() == 1:
                                    print(
                                        f"Lesson: {lesson['name']} (Grade {lesson['grade']}) - Instructor: {instructor}"
                                    )
                                    print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                    print(f"Type: {lesson['type']} (No classroom needed)\n")
    else:
        print("No optimal solution found.")

In [4]:
solve_schedule(data)


Optimal schedule found:

Lesson: Math101 (Grade 1) - Instructor: AEL
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: Biology101 (Grade 2) - Instructor: JaneSmith
Day: Tuesday, Time: 13:00 - 16:00
Type: Online (No classroom needed)

Lesson: English101 (Grade 4) - Instructor: AliceJohnson
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: LanguageLab303

Lesson: Physics201 (Grade 2) - Instructor: JohnDoe
Day: Tuesday, Time: 10:00 - 13:00
Type: FaceToFace, Classroom: ArtStudio302

Lesson: Philosophy101 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 8:00 - 9:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: History201 (Grade 2) - Instructor: AliceJohnson
Day: Monday, Time: 8:00 - 10:00
Type: Hybrid, Classroom: ComputerLab304

Lesson: Math102 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 12:00 - 15:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: Chemistry301 (Grade 3) - Instructor: LauraWilson
Day: Tuesday, Time: 8:0

In [16]:
def generate_schedule_pdf(solution_data, output_path="schedule.pdf"):
    """
    Generates a PDF schedule with grades as columns and days/times as rows.
    """
    css_content = """
        @page {
            size: A4 portrait;
            margin: 1cm;
        }
        
        body {
            font-family: Arial, sans-serif;
        }
        
        table {
            width: 100%;
            border-collapse: collapse;
            font-size: 8pt;
        }
        
        th, td {
            border: 1px solid black;
            padding: 1px;
            text-align: center;
            vertical-align: top;
            height: 12px;
        }
        
        td:empty {
            height: 8px;
            padding: 0;
        }
        
        .time-cell {
            width: 40px;
            background-color: #f0f0f0;
            font-size: 7pt;
            height: 12px;
            padding: 1px;
        }
        
        .grade-header {
            background-color: #e0e0e0;
            font-weight: bold;
            font-size: 9pt;
            height: 15px;
        }
        
        .course-cell {
            min-height: 15px;
            padding: 2px;
        }
        
        .face-to-face {
            background-color: #90EE90;
        }
        
        .online {
            background-color: #ADD8E6;
        }
        
        .hybrid {
            background-color: #FFB6C1;
        }
        
        .course-code {
            font-weight: bold;
            color: red;
            margin-bottom: 1px;
            font-size: 8pt;
        }
        
        .course-instructor {
            font-style: italic;
            font-size: 7pt;
        }
        
        .course-details {
            font-size: 7pt;
        }
        
        .title {
            text-align: center;
            font-size: 11pt;
            font-weight: bold;
            margin: 8px 0;
            background-color: yellow;
            padding: 3px;
        }
    """

    # Constants
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]
    times = list(range(8, 21))  # 8:00 to 20:00

    # Create schedule grid
    schedule_grid = {day: {grade: {time: None for time in times} for grade in grades} for day in days}

    # Populate schedule grid
    for lesson in solution_data:
        day = lesson["day"]
        grade = lesson["grade"]
        start_time = lesson["start_time"]
        for t in range(start_time, start_time + lesson["duration"]):
            schedule_grid[day][grade][t] = lesson

    # Generate HTML
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
    </head>
    <body>
        <div class="title">YTÜ Bilgisayar Mühendisliği Bölümü Ders Programı</div>
        <table>
            <tr>
                <th class="time-cell"></th>
    """

    # Add grade headers
    for grade in grades:
        html_content += f'<th class="grade-header">{grade}. Grade</th>'
    html_content += "</tr>"

    # Generate rows for each day and time
    for day in days:
        first_row_of_day = True
        for time in times:
            row_html = "<tr>"

            # Add time cell with day name if it's first row of the day
            if first_row_of_day:
                row_html += f'<td class="time-cell">{day}<br/>{time:02d}:00</td>'
                first_row_of_day = False
            else:
                row_html += f'<td class="time-cell">{time:02d}:00</td>'

            # Add cells for each grade
            for grade in grades:
                lesson = schedule_grid[day][grade][time]

                if lesson is None:
                    row_html += "<td></td>"
                elif time == lesson["start_time"]:
                    # Only create cell content at the start time
                    course_type_class = f"course-cell {lesson['type'].lower().replace(' ', '-')}"
                    row_html += f"""
                        <td rowspan="{lesson['duration']}" class="{course_type_class}">
                            <div class="course-code">{lesson['name']} ({lesson['duration']} hour)</div>
                            <div class="course-instructor">{lesson['instructor']}</div>
                            <div class="course-details">
                                ({lesson['type']})
                                {f"<br>{lesson['classroom']}" if lesson['classroom'] else ""}
                            </div>
                        </td>
                    """
                # Skip cell if it's covered by a rowspan from above

            row_html += "</tr>"
            html_content += row_html

    html_content += """
        </table>
    </body>
    </html>
    """

    # Create temporary files for HTML and CSS
    with tempfile.NamedTemporaryFile(suffix=".html", mode="w", encoding="utf-8", delete=False) as html_file:
        html_file.write(html_content)
        html_path = html_file.name

    with tempfile.NamedTemporaryFile(suffix=".css", mode="w", encoding="utf-8", delete=False) as css_file:
        css_file.write(css_content)
        css_path = css_file.name

    try:
        # Generate PDF
        HTML(html_path).write_pdf(output_path, stylesheets=[CSS(filename=css_path)])
    finally:
        # Clean up temporary files
        os.unlink(html_path)
        os.unlink(css_path)

In [17]:
def format_solver_solution(solver_output):
    """
    Convert solver output to the format needed by generate_schedule_pdf.

    Args:
        solver_output: List of strings containing the solver's output in the format:
        "Lesson: Math101 (Grade 1) - Instructor: AEL"
        "Day: Monday, Time: 8:00 - 11:00"
        "Type: FaceToFace, Classroom: Room101"

    Returns:
        List of dictionaries in the format needed by generate_schedule_pdf
    """
    formatted_data = []
    current_lesson = {}

    for line in solver_output:
        if line.startswith("Lesson:"):
            if current_lesson:
                formatted_data.append(current_lesson)
            current_lesson = {}
            # Parse lesson line
            lesson_parts = line.split(" - ")
            lesson_info = lesson_parts[0].replace("Lesson: ", "")
            name, grade = lesson_info.split(" (Grade ")
            current_lesson["name"] = name.strip()
            current_lesson["grade"] = int(grade.replace(")", ""))
            current_lesson["instructor"] = lesson_parts[1].replace("Instructor: ", "").strip()

        elif line.startswith("Day:"):
            # Parse day and time
            day_time = line.split(", ")
            current_lesson["day"] = day_time[0].replace("Day: ", "").strip()
            time_range = day_time[1].replace("Time: ", "").split(" - ")
            current_lesson["start_time"] = int(time_range[0].replace(":00", ""))
            current_lesson["duration"] = int(time_range[1].replace(":00", "")) - current_lesson["start_time"]

        elif line.startswith("Type:"):
            # Parse type and classroom
            type_info = line.split(", ")
            current_lesson["type"] = type_info[0].replace("Type: ", "").strip()
            if len(type_info) > 1:
                current_lesson["classroom"] = type_info[1].replace("Classroom: ", "").strip()
            else:
                current_lesson["classroom"] = None

    # Add the last lesson
    if current_lesson:
        formatted_data.append(current_lesson)

    return formatted_data

In [18]:
output = """
Lesson: Math101 (Grade 1) - Instructor: AEL
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: Biology101 (Grade 2) - Instructor: JaneSmith
Day: Tuesday, Time: 13:00 - 16:00
Type: Online (No classroom needed)

Lesson: English101 (Grade 4) - Instructor: AliceJohnson
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: LanguageLab303

Lesson: Physics201 (Grade 2) - Instructor: JohnDoe
Day: Tuesday, Time: 10:00 - 13:00
Type: FaceToFace, Classroom: ArtStudio302

Lesson: Philosophy101 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 8:00 - 9:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: History201 (Grade 2) - Instructor: AliceJohnson
Day: Monday, Time: 8:00 - 10:00
Type: Hybrid, Classroom: ComputerLab304

Lesson: Math102 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 12:00 - 15:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: Chemistry301 (Grade 3) - Instructor: LauraWilson
Day: Tuesday, Time: 8:00 - 11:00
Type: Online (No classroom needed)

Lesson: Spanish101 (Grade 2) - Instructor: EmmaThomas
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: ArtStudio302

Lesson: French101 (Grade 3) - Instructor: EmmaThomas
Day: Monday, Time: 8:00 - 10:00
Type: Hybrid, Classroom: ArtStudio302

Lesson: ComputerScience101 (Grade 4) - Instructor: MichaelChen
Day: Tuesday, Time: 8:00 - 10:00
Type: FaceToFace, Classroom: LanguageLab303

Lesson: Math103 (Grade 3) - Instructor: MichaelChen
Day: Tuesday, Time: 11:00 - 13:00
Type: Online (No classroom needed)

Lesson: Art101 (Grade 1) - Instructor: SarahPatel
Day: Monday, Time: 10:00 - 12:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: History202 (Grade 4) - Instructor: SarahPatel
Day: Monday, Time: 8:00 - 10:00
Type: Hybrid, Classroom: LanguageLab303

Lesson: Music101 (Grade 1) - Instructor: DavidKim
Day: Tuesday, Time: 11:00 - 12:00
Type: FaceToFace, Classroom: MusicRoom301

Lesson: Drama101 (Grade 2) - Instructor: DavidKim
Day: Tuesday, Time: 8:00 - 10:00
Type: FaceToFace, Classroom: ArtStudio302

"""

In [19]:
formatted_data = format_solver_solution(output.split("\n"))

In [23]:
generate_schedule_pdf(formatted_data, "scheduleNEW.pdf")